In [1]:
import skbio as skb
import numpy as np
import pandas as pd
import seaborn as sns
from ecopy import Mantel

from matplotlib import pyplot as plt
import pylab as pl
from skbio import TreeNode
from skbio.stats.distance import mantel
from scipy.stats import linregress
from scipy.spatial.distance import squareform, pdist
from os.path import abspath, join
import os.path
from os import makedirs
from Bio import Phylo
from Bio.Phylo.Consensus import *

In [2]:
tree_dir = abspath('/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged')
host_tree_fp = join(tree_dir, 'host_tree_list.nwk')
host_tree_orig= TreeNode.read(host_tree_fp, format='newick', convert_underscores=False)

host_tips_orig = [x.name for x in host_tree_orig.tips()]

In [3]:
## read in metadata
md_dir = '/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/metadata'
host_md_fp = join(md_dir, 'vert_metadata_8_23.txt')
host_md_orig = pd.read_csv(host_md_fp, sep='\t', encoding='windows-1252')
print(host_md_orig.shape)

## filter metadata based on available value from timetree! 
## Five samples were lost 
## These hosts were missing from timetree
#(Geospiza acutirostris (2 samples), Aspius aspius (2 samples), Cervus canadensis canadensis)
host_md_TT = host_md_orig.loc[(host_md_orig['TimeTree_returned'].isin(host_tips_orig))]

# filter to just these host classes
include_classes = ['Amphibia',
                    'Mammalia',
                    'Aves',
                    'Reptilia',
                  'Hyperoartia',
                  'Actinopterygii']

host_md_TT_classes = host_md_TT.loc[host_md_TT['host_class'].isin(include_classes)]
print(host_md_TT_classes.shape)

host_md_TT_classes_wild= host_md_TT_classes.loc[host_md_TT_classes['captive_wild']=="wild"]
print(host_md_TT_classes_wild.shape)

(671, 30)
(657, 30)
(500, 30)


In [6]:
## Strain trees we hope to investigate 

tree1 = "/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/output_20_B_vulgatus_test/RAxML_bestTree.s__Bacteroides_vulgatus.StrainPhlAn3.tre"
tree2 = "/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/output_20_B_vulgatus_test/GAMMA/RAxML_bestTree.s__Bacteroides_vulgatus.StrainPhlAn3_GAMMA.tre"

paths = [tree1, tree2]

In [5]:
one_per_sp = True ## One sample per species? 
number_of_iterations=100 ## Number of iteration to create the sample tree given one_per_sp
correlation_coefficient = "spearman"
marker_threshold="20"
sample_group= ""## "" or "_mammallianHost" or "_nonMammallianHost"

In [8]:
mantel_results_dict={}
print( "-----The mantel test for strain trees built with markers threshold at " + marker_threshold +"%-----")
for path in paths:
    
    print( "\n \n Investigating tree at" + path +":" )
    host_md=host_md_TT_classes #### !!!!!!!!!!!!!!! Change this metadata according to the md desired
    host_tree=host_tree_orig
    host_tips=host_tips_orig

    #sample_tree_fp = join(tree_dir, 'strain_tree_GTR_200reps.nwk')
    sample_tree_fp = path
    try:
        sample_tree= TreeNode.read(sample_tree_fp, format='newick', convert_underscores=False)
    except FileNotFoundError:
        print("No sample tree available")
        continue

    sample_tips = [x.name for x in sample_tree.tips()]

    ## Filter metadata based on the overlapping host tree and sample tree
    md_sampleID_ids = set(host_md['SampleID2'])
    md_timeTree_ids = set(host_md['TimeTree_returned'])


    sample_tips_ids=set(sample_tips)
    host_tips_ids=set(host_tips)

    shared_sample_ids=sample_tips_ids & md_sampleID_ids
    shared_host_ids=host_tips_ids & md_timeTree_ids

        
#     print("Filtering metadata with shared_ids......")
    host_md = host_md.loc[(host_md['SampleID2'].isin(shared_sample_ids))]
    host_md = host_md.loc[(host_md['TimeTree_returned'].isin(shared_host_ids))]
    
    if len(host_md) <=3: 
        print("Too few samples remaining (<=3) aftering filtering by sample name and TimeTree_returned name, skipping...")
        continue
    


    md_sampleID_ids = set(host_md['SampleID2'])
    md_timeTree_ids = set(host_md['TimeTree_returned'])

    # Prune host tree 

    host_tree = host_tree.shear(host_md['TimeTree_returned'])
    host_tips = [x.name for x in host_tree.tips()]

        
    # Prune sample tree randomly multiple times and save into list! 
    sample_tree_list=[]
    if one_per_sp:
        for i in range(0, number_of_iterations):
            ## Find one sample per species randomly
            host_md_subset =  host_md.loc[(host_md['SampleID2'].isin(sample_tips)),].groupby('TimeTree_returned').sample(1).reset_index()
            
            if len(host_md_subset) <=3: 
                break
            
            ## Shear sample tree
            sheared_sample_tree = sample_tree.shear(host_md_subset['SampleID2'])
            ## Add sheared sample tree to list
            sample_tree_list.append(sheared_sample_tree)


    ## Check if number of tips in sample and host tree is the same 
#     print("Same number of nodes between the trees? ==> ")
#     print(len(one_sample_tips) == len(host_tips))
#     one_sample_tips = [x.name for x in sample_tree_list[0].tips()]
    ## Check if the number of sample tree is the same as number of iterations
    if len(sample_tree_list) != number_of_iterations:
        print("Too few samples remaining (<=3) aftering filtering by one sample per species, skipping...")
        continue
    
    ## Calculate patristic distances
    host_patristic_dm = host_tree.tip_tip_distances()

#     sample_tree_list[9]

    sample_patristic_dm_list=[]
    for i in range(0, number_of_iterations):
        sample_patristic_dm = sample_tree_list[i].tip_tip_distances()

        ## acquire sample tip from each tree
        sample_tips = [x.name for x in sample_tree_list[i].tips()]

        ## Change label of sample dm 
        ## get corresponding name of the ids
        rename_df = host_md.loc[host_md['SampleID2'].isin(sample_tips),
                                     ['SampleID2','TimeTree_returned']].set_index('SampleID2')
        rename = [rename_df.loc[x, 'TimeTree_returned'] for x in sample_tips]
        ## Change label of distance matrix 
        sample_patristic_dm.ids = rename

        ## add to host_patristic_dm
        sample_patristic_dm_list.append(sample_patristic_dm)

    ## Test two dm.ids are the same 
#     print("Are the ids of the distance matrices the same? ==> ")
#     print(set(host_patristic_dm.ids) == set(sample_patristic_dm_list[0].ids))

    # Mantel test with results averaged over iterations set
    sum_corr=0
    sum_p=0
    sum_group=0
    for i in range(0, number_of_iterations):
        corr, p, n = mantel(host_patristic_dm, sample_patristic_dm_list[i], method=correlation_coefficient )
        sum_corr+=corr
        sum_p+=p
        sum_group+=n
    print("The average correlation, p values over "+ str(number_of_iterations) + " iterations are:")
    aver_corr=sum_corr/number_of_iterations
    aver_p=sum_p/number_of_iterations
    aver_group=sum_group/number_of_iterations

    print("Number of samples\t"+"average "+ correlation_coefficient + " coorelation\t"+"average p-value\t")
    print(str(aver_group)+"\t"+str(aver_corr)+"\t"+str(aver_p))
    

-----The mantel test for strain trees built with markers threshold at 20%-----

 
 Investigating tree at/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/output_20_B_vulgatus_test/RAxML_bestTree.s__Bacteroides_vulgatus.StrainPhlAn3.tre:
The average correlation, p values over 100 iterations are:
Number of samples	average spearman coorelation	average p-value	
30.0	0.3553937101608195	0.005050000000000003

 
 Investigating tree at/panfs/panfs1.ucsd.edu/panscratch/jhc103/VertMetaphlan-frmerged/output_20_B_vulgatus_test/GAMMA/RAxML_bestTree.s__Bacteroides_vulgatus.StrainPhlAn3_GAMMA.tre:
The average correlation, p values over 100 iterations are:
Number of samples	average spearman coorelation	average p-value	
30.0	0.354618190128134	0.004950000000000003
